<a href="https://colab.research.google.com/github/Ucchwas/ECG-Classification-using-ML-models/blob/main/Hybrid_Multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [181]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
from sklearn.utils import class_weight
import warnings
warnings.filterwarnings('ignore')

In [182]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/mitbih_train_Ihua.csv')

In [183]:
df.drop('1',axis='columns',inplace=True)
df.fillna(0, inplace=True)
X = df.drop('Label1',axis='columns')
y = testLabels = df.Label1.astype(np.float32)

In [184]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2,train_size=0.8, random_state=15, stratify=y)
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [185]:
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, LSTM, BatchNormalization
from keras.models import Model
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report
from keras.layers import concatenate

In [186]:
def CNN_RNN(X_train, y_train, X_test, y_test, loss, weights):
    im_shape = (X_train.shape[1], 1)
    inputs_cnn = Input(shape=im_shape, name='inputs_cnn')

    # Convolutional layers
    conv1_1 = Conv1D(64, 6, activation='relu', input_shape=im_shape)(inputs_cnn)
    conv1_1 = BatchNormalization()(conv1_1)
    pool1 = MaxPooling1D(pool_size=3, strides=2, padding="same")(conv1_1)

    conv2_1 = Conv1D(64, 3, activation='relu')(pool1)
    conv2_1 = BatchNormalization()(conv2_1)
    pool2 = MaxPooling1D(pool_size=2, strides=2, padding="same")(conv2_1)

    conv3_1 = Conv1D(64, 3, activation='relu')(pool2)
    conv3_1 = BatchNormalization()(conv3_1)
    pool3 = MaxPooling1D(pool_size=2, strides=2, padding="same")(conv3_1)

    flatten = Flatten()(pool3)
    # Reshape the output of Flatten to be 3D
    reshaped = Reshape((1, -1))(flatten)  # Reshape to (batch_size, 1, features)


    # Recurrent layer (LSTM)
    lstm_layer = LSTM(64, return_sequences=False)(reshaped)

    # Fully connected layers
    dense_end1 = Dense(64, activation='relu')(lstm_layer)
    dense_end2 = Dense(32, activation='relu')(dense_end1)
    main_output = Dense(5, activation='sigmoid', name='main_output')(dense_end2)

    model = Model(inputs=inputs_cnn, outputs=main_output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    if weights == -1:
        model.fit(X_train, y_train, epochs=10)
    else:
        model.fit(X_train, y_train, epochs=10, class_weight=weights)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_train)
    y_preds = (y_preds > 0.5).astype(int)

    print("Classification Report: \n", classification_report(y_train, y_preds))
    return model, y_preds


In [187]:
from keras.utils import to_categorical

# Convert y_train to one-hot encoding
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [188]:
model,history=CNN_RNN(X_train,y_train,X_test,y_test, 'binary_crossentropy', -1)

Epoch 1/10
2189/2189 [==============================] - 89s 38ms/step - loss: 0.0542 - accuracy: 0.9592
Epoch 2/10
2189/2189 [==============================] - 89s 41ms/step - loss: 0.0305 - accuracy: 0.9772
Epoch 3/10
2189/2189 [==============================] - 85s 39ms/step - loss: 0.0259 - accuracy: 0.9792
Epoch 4/10
2189/2189 [==============================] - 82s 38ms/step - loss: 0.0221 - accuracy: 0.9826
Epoch 5/10
2189/2189 [==============================] - 90s 41ms/step - loss: 0.0191 - accuracy: 0.9851
Epoch 6/10
2189/2189 [==============================] - 84s 38ms/step - loss: 0.0174 - accuracy: 0.9861
Epoch 7/10
2189/2189 [==============================] - 89s 41ms/step - loss: 0.0162 - accuracy: 0.9871
Epoch 8/10
2189/2189 [==============================] - 85s 39ms/step - loss: 0.0146 - accuracy: 0.9880
Epoch 9/10
2189/2189 [==============================] - 83s 38ms/step - loss: 0.0131 - accuracy: 0.9893
Epoch 10/10
548/548 [==============================] - 6s 10ms/s

In [189]:
model.save("Hybrid_Multiclass.h5")

In [190]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model("Hybrid_Multiclass.h5")

In [191]:
run_model = tf.function(lambda x: model(x))
# This is important, let's fix the input size.
BATCH_SIZE = 1
STEPS = 186
INPUT_SIZE = 1
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "keras_lstm"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()
open("Hybrid_Multiclass.tflite", "wb").write(tflite_model)

1647460